In [ ]:
import numpy
import random

import pyibm

from helper import *

%matplotlib inline

In [ ]:
pyibm.__version__

In [ ]:
delta_kernel = pyibm.delta_roma_et_al_1999
delta_kernel_size = 3

In [ ]:
config = dict(x=dict(start=-2.0, end=2.0, num_cells=20),
              y=dict(start=-2.0, end=2.0, num_cells=200))
grid = pyibm.GridBase(config=config)
grid

In [ ]:
gridc = pyibm.GridCellCentered(grid=grid)
gridx = pyibm.GridFaceX(grid=grid)
gridy = pyibm.GridFaceY(grid=grid)

In [ ]:
radius = 0.5
xc, yc = 0.0, 0.0
ds = gridx.x.get_widths()[0]
N = int(round(2 * numpy.pi * radius / ds))
N = 50
theta = numpy.linspace(0.0, 2 * numpy.pi, num=N + 1)[:-1]
x, y = xc + radius * numpy.cos(theta), yc + radius * numpy.sin(theta)
body = pyibm.Body(x, y, grid=gridc)
body

In [ ]:
Op = pyibm.assemble_delta(body, gridc, gridx, gridy,
                          kernel=delta_kernel,
                          kernel_size=delta_kernel_size)

In [ ]:
R = pyibm.assemble_R(gridx, gridy)
MHat = pyibm.assemble_MHat(gridx, gridy)
E = Op @ R @ MHat

In [ ]:
S = pyibm.assemble_surfaces(body)
H = Op.T @ S

In [ ]:
Fx = numpy.random.rand(body.size)
Fy = numpy.random.rand(body.size)
F = numpy.empty(body.ndim * body.size)
F[::body.ndim], F[1::body.ndim] = Fx, Fy

f = H @ F
fx = f[:gridx.size].reshape(gridx.shape)
fy = f[gridx.size:].reshape(gridy.shape)

dS = body.ds
dx = gridc.x.get_widths()[gridc.ijk(body.neighbors[0])[0]]
dy = gridc.y.get_widths()[gridc.ijk(body.neighbors[0])[1]]
dV = dx * dy

print(numpy.sum(Fx) * dS, numpy.sum(Fy) * dS)
print(numpy.sum(fx) * dV, numpy.sum(fy) * dV)

$$
\sum_{i, j, k} \mathbf{f}\left( \mathbf{x}_{i, j, k} \right) \Delta v_{i, j, k} = \sum_{m} \mathbf{F}\left( \mathbf{X}_{m} \right) \Delta V_{m}
$$

In [ ]:
Xx, Yx = numpy.meshgrid(gridx.x.vertices, gridx.y.vertices)
Xy, Yy = numpy.meshgrid(gridy.x.vertices, gridy.y.vertices)

print((numpy.sum(Xy * fy) - numpy.sum(Yx * fx)) * dV)
print(numpy.sum(body.x * Fy - body.y * Fx) * dS)

$$
\sum_{i j, k} \mathbf{x}_{i j, k} \times \mathbf{f}\left( \mathbf{x}_{i, j, k} \right) \Delta v_{i, j, k} = \sum_{m} \mathbf{X}_{m} \times \mathbf{F}\left( \mathbf{X}_{m} \right) \Delta V_{m}
$$

In [ ]:
plot_contourf(fx, gridx, body, show_grid=False);

In [ ]:
xstart, xend = 0.0, 1.0
nx = 100
x = numpy.linspace(xstart, xend, num=nx + 1)
dx = (xend - xstart) / nx

tol = 1e-14
for _ in range(20):
    xb = random.uniform(xstart + 5 * dx, xend - 5 * dx)
    res1 = dx * numpy.sum([delta_kernel(abs(xi - xb), dx)
                           for xi in x])
    res2 = dx * numpy.sum([(xi - xb) * delta_kernel(abs(xi - xb), dx)
                           for xi in x])
    assert abs(res1 - 1.0) <= tol
    assert abs(res2 - 0.0) <= tol

$$
\sum_{i j, k} \delta_{h}\left( \mathbf{X}_{i, j, k} - \mathbf{X} \right) \Delta v_{i, j, k} = 1
$$

$$
\sum_{i j, k}\left( \mathbf{x}_{i, j, k} - \mathbf{X} \right) \delta_{h}\left( \mathbf{x}_{i j, k} - \mathbf{X} \right) \Delta v_{i, j, k} = 0
$$